# Econometric Feature Selection Using Lasso

In [11]:
#imports 
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as sm
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange, tqdm
import statsmodels.api as sm
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

In [2]:
class CreateFeatures:
    """
    We define a class which builds the feature dataframe 
    """
    
    def __init__(self, year = 1962, data_dir = "data/"):
        self.year = year
        self.data_dir = data_dir
        
    def prepare_econ_features(self, filter_gdp = True):
        
        #DATA IMPORT
        #import dictionary with all features from WB
        with open(self.data_dir + 'all_wb_indicators.pickle', 'rb') as handle:
            features_dict = pkl.load(handle)
            
        self.feature_list = list(features_dict.keys())[1:]
        #import list of all features we want to select for

        #look up each of the features -- add country feature in that year 
        i = 0
        for feature in self.feature_list:
            #find dataframe corresponding to specific feature name
            df = features_dict[feature]
            
            if (i == 0):
                self.features = df[["economy", "YR" + str(self.year)]]
            else: 
                self.features = pd.merge(self.features, 
                                            df[["economy", "YR" + str(self.year)]],
                                            on = "economy", how = "outer")
            self.features.rename(columns = {"YR" + str(self.year): feature}, inplace = True)
            i = i+1
        
        #prepare GDP feature
        self.gdp_growth = features_dict['NY.GDP.MKTP.KD.ZG']
        cols = list(self.gdp_growth.columns.copy())
        cols.remove("economy")
        self.gdp_growth["country_sd"] = self.gdp_growth[cols].std(axis=1)
        #select potential variables 
        self.gdp_growth["prev_gdp_growth"] = self.gdp_growth["YR" + str(self.year-1)]
        self.gdp_growth["current_gdp_growth"] = self.gdp_growth["YR" + str(self.year)] 
        self.gdp_growth["future_3y_gdp_growth"] = self.gdp_growth["YR" + str(self.year+1)]\
        +self.gdp_growth["YR" + str(self.year+2)]+self.gdp_growth["YR" + str(self.year+3)] 
        #we eliminate countries that are too volatile in growth -- probably an indicator that growth estimates are inaccurate
        self.high_vol_countries = list(set(self.gdp_growth[self.gdp_growth["country_sd"] > 10]["economy"]))
        self.gdp_growth = self.gdp_growth[["economy", "prev_gdp_growth",
                                "current_gdp_growth", "future_3y_gdp_growth"]].dropna()
        
        #combine GDP and other features
        self.features = pd.merge(self.gdp_growth, self.features,
                                   on = "economy", how = "left")
        #we only keep countries where we observe GDP growth -- otherwise nothing to predict
        #we keep countries where other features may be missing -- and fill NAs with 0 
        self.features.rename(columns = {"economy": "country_code"}, inplace = True)
        
    def prepare_network_features(self):
        """
        We create an initial, import-centric trade link pandas dataframe for a given year
        """
        #get product codes
        data_dict = pd.read_json('https://comtrade.un.org/data/cache/classificationS2.json')
        data_cross = []
        i = 0
        for item_def in list(data_dict["results"]):
            if(i >= 2):
                data_cross.append(item_def["text"].split(" - ", 1))
            i = i+1

        self.product_codes = pd.DataFrame(data_cross, columns = ['code', 'product'])
        self.product_codes["sitc_product_code"] = self.product_codes["code"]
        
        #get country codes
        self.country_codes = pd.read_excel(self.data_dir + "ISO3166.xlsx")
        self.country_codes["location_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["partner_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["country_i"] = self.country_codes["English short name"]
        self.country_codes["country_j"] = self.country_codes["English short name"]
        
        #get trade data for a given year
        trade_data = pd.read_stata(self.data_dir + "country_partner_sitcproduct4digit_year_"+ str(self.year)+".dta") 
        #merge with product / country descriptions
        trade_data = pd.merge(trade_data, self.country_codes[["location_code", "country_i"]],on = ["location_code"])
        trade_data = pd.merge(trade_data, self.country_codes[["partner_code", "country_j"]],on = ["partner_code"])
        trade_data = pd.merge(trade_data, self.product_codes[["sitc_product_code", "product"]], 
                              on = ["sitc_product_code"])
        ###select level of product aggregation
        trade_data["product_category"] = trade_data["sitc_product_code"].apply(lambda x: x[0:1])
        #trade_data = trade_data[trade_data["product_category"] == "1"]
        
        #keep only nodes that we have features for
        trade_data = trade_data[trade_data["location_code"].isin(self.features["country_code"])]
        trade_data = trade_data[trade_data["partner_code"].isin(self.features["country_code"])]
        
        if (len(trade_data.groupby(["location_code", "partner_code", "sitc_product_code"])["import_value"].sum().reset_index()) != len(trade_data)):
            print("import, export, product combination not unique!")
        self.trade_data1 = trade_data
        #from import-export table, create only import table
        #extract imports
        imports1 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports1 = imports1[imports1["import_value"] != 0]
        #transform records of exports into imports
        imports2 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports2["temp1"] = imports2['partner_code']
        imports2["temp2"] = imports2['location_code']

        imports2['location_code'] = imports2["temp1"]
        imports2['partner_code'] = imports2["temp2"]
        imports2["import_value"] = imports2["export_value"]
        imports2 = imports2[imports2["import_value"] != 0]
        imports2 = imports2[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        
        imports_table = pd.concat([imports1, imports2]).drop_duplicates()
        
        #rename columns for better clarity
        imports_table["importer_code"] = imports_table["location_code"]
        imports_table["exporter_code"] = imports_table["partner_code"]
        imports_table["importer_name"] = imports_table["country_i"]
        imports_table["exporter_name"] = imports_table["country_j"]
        
        cols = ["importer_code", "exporter_code", "importer_name", "exporter_name",
               'product_id', 'year', 'import_value', 'sitc_eci', 'sitc_coi',
               'sitc_product_code', 'product', "product_category"]
        imports_table = imports_table[cols]
        
        exporter_total = imports_table.groupby(["exporter_code"])["import_value"].sum().reset_index()
        exporter_total = exporter_total.rename(columns = {"import_value": "export_total"})
        
        importer_total = imports_table.groupby(["importer_code"])["import_value"].sum().reset_index()
        importer_total = importer_total.rename(columns = {"import_value": "import_total"})
        
        ##### COMPUTE CENTRALITY FOR COUNTRY
        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        imports_table_grouped = pd.merge(imports_table_grouped, importer_total, on = "importer_code")
        imports_table_grouped["import_fraction"] = imports_table_grouped["import_value"]\
                        /imports_table_grouped["import_total"]*100
        
        self.trade_data = imports_table_grouped
        
        #filter features and nodes to ones that are connected to others in trade data
        list_active_countries = list(set(list(self.trade_data ["importer_code"])+\
                        list(self.trade_data ["exporter_code"])))
        self.features = self.features[self.features["country_code"].isin(list_active_countries)].reset_index()
        self.features["node_numbers"] = self.features.index
        
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", create_using = nx.DiGraph())
        
        self.G = G
        self.centrality_overall= nx.eigenvector_centrality(G, max_iter= 10000) 
        self.centrality_overall = pd.DataFrame(list(map(list, self.centrality_overall.items())), 
                                               columns = ["country_code", "centrality_overall"])
        G=nx.from_pandas_edgelist(self.trade_data, 
                          "exporter_code", "importer_code", ["import_fraction"])
        weighted_centrality = nx.eigenvector_centrality(G, weight = "import_fraction", max_iter= 10000) 
        weighted_centrality  = pd.DataFrame(list(map(list, weighted_centrality.items())), 
                                               columns = ["country_code", "weighted_centrality"])
        self.centrality_overall = pd.merge(self.centrality_overall, weighted_centrality, on = "country_code")
        
                               
        ##### COMPUTE CENTRALITY FOR COUNTRY IN PRODUCT CATEGORIES

        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        products_grouped = imports_table.groupby(["product_category"])["import_value"].sum().reset_index()
        products_grouped = products_grouped.rename(columns = {"import_value": "import_product_total"})
        
        #sum exports in each category 
        self.export_types = imports_table.groupby(["importer_code", "exporter_code", "product_category"])["import_value"].sum().reset_index()
        self.export_types = pd.merge(products_grouped, self.export_types, on = "product_category")
        
        self.export_types["product_export_fraction"] = self.export_types["import_value"]\
                                                    /self.export_types["import_product_total"]*100
        
        list_products = list(set(self.export_types["product_category"]))
        
        i = 0 
        for product in list_products:
            
            temp = self.export_types[self.export_types["product_category"] == product].copy()
            
            G_w=nx.from_pandas_edgelist(temp, 
                "exporter_code", "importer_code", ["product_export_fraction"], create_using = nx.DiGraph())
            centrality_product_w = nx.eigenvector_centrality(G_w, weight = "product_export_fraction", 
                                                           max_iter= 10000)

            G=nx.from_pandas_edgelist(temp,"exporter_code", "importer_code", create_using = nx.DiGraph())
            centrality_product = nx.eigenvector_centrality(G,max_iter= 10000)

            if(i == 0):
                self.centrality_product = pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product])
                

            else: 
                self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product.items())), 
                                               columns = ["country_code", "prod_" + product]), 
                                                  on = "country_code")
                
            self.centrality_product = pd.merge(self.centrality_product, 
                                               pd.DataFrame(list(map(list, centrality_product_w.items())), 
                                               columns = ["country_code", "prod_w_" + product]), 
                                                  on = "country_code")
            
            i = i+1         
    
    def combine_normalize_features(self):
        
        self.combined_features = pd.merge(self.features, self.centrality_overall,on = "country_code")
        self.combined_features = pd.merge(self.combined_features, self.centrality_product,on = "country_code")
        #step eliminates NA and nodes that are not in graph, since they will have NA for graph features
        self.combined_features = self.combined_features.drop(columns = ["index"])
        #filter to non-volatile countries
        self.combined_features = self.combined_features[\
                                ~self.combined_features.country_code.isin(self.high_vol_countries)]
        #filter both trade data and features data to same subset of countries
        self.combined_features = self.combined_features[\
                                self.combined_features.country_code.isin(self.trade_data.importer_code)|\
                                self.combined_features.country_code.isin(self.trade_data.exporter_code)]
        self.trade_data = self.trade_data[\
                          self.trade_data.importer_code.isin(self.combined_features.country_code)&\
                          self.trade_data.exporter_code.isin(self.combined_features.country_code)]
        
        #extract categorical features and dependent variable for growth
        self.current_growth_list = list(self.combined_features["current_gdp_growth"])
        self.future_growth_list = list(self.combined_features["future_3y_gdp_growth"])
        self.combined_features["current_growth_cat"] = self.combined_features["current_gdp_growth"].\
                                                        apply(lambda x: growth_categories(x, self.current_growth_list))
        self.combined_features["future_growth_cat"] = self.combined_features["future_3y_gdp_growth"].\
                                                        apply(lambda x: growth_categories(x, self.future_growth_list))
        
        features_to_norm = list(self.combined_features.columns.copy())
        non_norm = ["country_code",'current_growth_cat','future_growth_cat', "node_numbers"]
        cols_insufficient_data = list(self.combined_features.loc[:, self.combined_features.nunique() < 2].columns.copy())
        non_norm.extend(cols_insufficient_data)
 
        features_to_norm = [x for x in features_to_norm if x not in non_norm]
        scaler = StandardScaler()
        #we preserve NAs in the scaling
        self.combined_features[features_to_norm] = scaler.fit_transform(self.combined_features[features_to_norm])
        self.combined_features.fillna(0, inplace = True) #we fill NA after scaling 
        #check that feature has at least 20% coverage in a given year -- otherwise set to NA
        for feature in self.feature_list:
            coverage = len(self.combined_features[self.combined_features[feature] != 0])/len(self.combined_features)
            if(coverage <= 0.20): self.combined_features[feature] = 0

In [3]:
with open('Trade_exploration/trade_data.pickle', 'rb') as handle:
    data_dict = pkl.load(handle)

In [39]:
for year in data_dict:
    data_dict[year].combined_features['year'] = year

/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_35843/1524783330.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_dict[year].combined_features['year'] = year
/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_35843/1524783330.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_dict[year].combined_features['year'] = year
/var/folders/qh/0wyfgfcs3399gr_m2wrrtnzc0000gn/T/ipykernel_35843/1524783330.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

In [78]:
combined_df = data_dict[1990].combined_features

for year in range(1990, 2019):
    combined_df = pd.concat([combined_df, data_dict[year].combined_features])

In [79]:
year_col = combined_df['year']
brexit = []
for i in year_col:
    if i<2016:
        brexit.append(0)
    else:
        brexit.append(1)

In [82]:
non_features = ['EG.ELC.ACCS.ZS', 'EG.FEC.RNEW.ZS', 'MS.MIL.TOTL.TF.ZS',
       'SL.UEM.TOTL.ZS', "country_code", "const", "prev_gdp_growth","current_gdp_growth","future_3y_gdp_growth", "current_growth_cat","future_growth_cat","year"]
feat_columns = [col for col in list(combined_df.columns) if col not in non_features]

In [83]:
X = combined_df[feat_columns]
X = sm.add_constant(X)
Y = brexit

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [84]:
# parameters to be tested on GridSearchCV
params = {"alpha":np.arange(0.00001, 10, 500)}

# Number of Folds and adding the random state for replication
kf=KFold(n_splits=5,shuffle=True, random_state=42)

# Initializing the Model
lasso = Lasso()

# GridSearchCV with model, params and folds.
lasso_cv=GridSearchCV(lasso, param_grid=params, cv=kf)
lasso_cv.fit(X_train, y_train)
print("Best Params {}".format(lasso_cv.best_params_))

/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.965e+01, tolerance: 2.829e-02
  model = cd_fast.enet_coordinate_descent(
/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.062e+01, tolerance: 2.673e-02
  model = cd_fast.enet_coordinate_descent(
/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

Best Params {'alpha': 1e-05}


/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.173e+02, tolerance: 3.461e-02
  model = cd_fast.enet_coordinate_descent(


In [85]:
# calling the model with the best parameter
lasso1 = Lasso(alpha=lasso_cv.best_params_['alpha'])
lasso1.fit(X_train, y_train)

# Using np.abs() to make coefficients positive.
lasso1_coef = np.abs(lasso1.coef_)

names=X_train.columns

# Subsetting the features which has more than 0.001 importance.
feature_subset=np.array(names)[lasso1_coef>0.001]

/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.173e+02, tolerance: 3.461e-02
  model = cd_fast.enet_coordinate_descent(


In [86]:
X_train = X_train[feature_subset]
X_test = X_test[feature_subset]

In [87]:
X_train

,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CREL.HA,AG.LND.CROP.ZS,AG.LND.EL5M.RU.K2,...,prod_6,prod_w_6,prod_1,prod_w_1,prod_4,prod_w_4,prod_7,prod_w_7,prod_2,prod_w_2
169,0.000000,-0.279209,-0.330270,-1.403446,-0.233606,0.236460,-0.602872,-0.274581,-0.583312,0.00000,...,1.247002,0.107595,1.176298,0.381088,1.207157,0.813162,1.326765,0.025638,0.942449,-0.032664
25,0.000000,-0.199725,0.165362,-0.188574,-0.177829,0.675096,-0.813427,-0.290387,-0.530957,0.00000,...,-0.380713,-0.319738,-0.406570,-0.321306,-0.488288,-0.362811,-0.408224,-0.310418,-0.498671,-0.311677
30,0.000000,-0.369551,-0.024114,-1.129407,-0.040696,-0.207089,-0.786083,-0.186754,-0.544899,-0.38631,...,-0.420007,-0.339532,-0.730559,-0.336850,-0.591661,-0.402941,-0.334412,-0.355532,-0.547192,-0.325859
24,0.000000,0.000000,-0.384966,-1.637327,-0.337774,-0.903715,-0.993164,-0.352655,-0.436777,0.00000,...,-0.411490,-0.367302,-0.083634,-0.319838,-0.260770,-0.365332,-0.468529,-0.332921,-0.344792,-0.358135
123,0.000000,-0.218880,0.210941,0.577790,-0.100131,0.128385,-0.564866,-0.146141,-0.515386,0.00000,...,0.063917,-0.380640,-0.724668,-0.334659,-0.010350,-0.315406,0.476118,-0.344302,-0.012871,-0.317351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.000000,0.192855,-0.376983,-0.950914,-0.347905,-0.794110,-1.003424,-0.342616,-0.528567,-0.36003,...,-0.451501,-0.363062,-0.340638,-0.334186,-0.348795,-0.361676,-0.504238,-0.300399,-0.281869,-0.281664
63,0.000000,0.000000,-0.392109,-1.665857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,-0.943546,-0.363232,-0.734649,-0.352862,-1.228760,-0.395932,-0.934872,-0.397078,-1.023276,-0.352036
53,0.000000,-0.044789,-0.263965,-0.338334,-0.280562,-0.534896,-0.684050,-0.292175,0.061449,0.00000,...,-0.159051,-0.312424,-0.216908,-0.319075,-0.280049,-0.307520,-0.056781,-0.282004,-0.123900,-0.284097
5,-0.217937,1.419020,-0.369742,-1.502996,-0.348598,-0.851303,-1.027185,-0.354519,-0.500453,0.00000,...,1.572465,0.650149,1.867790,0.384771,1.585214,-0.148643,1.365299,0.506813,1.615818,-0.074545
